In [ ]:
!pip uninstall -q keras==2.4.3

Proceed (y/n)? y


In [ ]:
!pip install -q keras==2.3.1
import keras

     |████████████████████████████████| 378kB 2.8MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 


Using TensorFlow backend.


In [ ]:
!pip install tensorflow==2.0.0

     |████████████████████████████████| 86.3MB 47kB/s 
     |████████████████████████████████| 450kB 45.0MB/s 
     |████████████████████████████████| 3.8MB 39.4MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=aa7e71e151381eddaf1e57671ef364f701a2d78dc6e69ef6d3fa5d4357a3a7df
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: t

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import glob, os
import pandas as pd
import keras
import numpy as np
np.random.seed(1337)  # for reproducibility
########################
from tqdm import tqdm
from random import shuffle as shuffle_array
from sklearn.model_selection import StratifiedKFold
########################
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,  Convolution1D,Convolution2D, MaxPooling2D , AveragePooling2D
from keras.utils import np_utils
from keras.layers.core import Reshape
########################

In [ ]:
df_data=pd.read_csv('/content/drive/My Drive/datatrain-ban-truong-phuong-hop.csv')
print(df_data.shape)
df_data

(62046, 10)


,time,a-x,a-y,a-z,a-mag,g-x,g-y,g-z,g-mag,id
0,1.600000e+12,-0.442927,2.410961,9.539697,9.849606,0.128453,-0.212825,0.222550,0.333652,0
1,1.600000e+12,-0.495600,2.401384,9.557653,9.867168,0.156553,-0.298957,0.188953,0.386765,0
2,1.600000e+12,-0.377087,2.363077,9.518148,9.814350,0.157774,-0.308120,0.177957,0.389230,0
3,1.600000e+12,-0.458490,2.372654,9.424775,9.729650,0.168159,-0.283075,0.190175,0.380231,0
4,1.600000e+12,-0.354342,2.395399,9.392453,9.699570,0.141281,-0.202441,0.203614,0.320002,0
...,...,...,...,...,...,...,...,...,...,...
62041,1.600000e+12,-0.231000,-8.614000,0.741000,8.648898,0.475841,-1.104531,0.366686,1.257327,3
62042,1.600000e+12,-1.021000,-8.748000,2.372000,9.121202,0.259841,-0.815531,0.018686,0.856129,3
62043,1.600000e+12,-1.462000,-8.578000,2.544000,9.065951,-0.073159,-0.433531,-0.128314,0.458002,3
62044,1.600000e+12,-1.405000,-8.533000,1.826000,8.838574,-0.088159,-0.367531,-0.215314,0.434984,3


In [ ]:
Ns = 200 #size of each of contiguous chunk
N_samples=df_data.shape[0]//Ns
rounded_data_idx = N_samples*Ns

col_sensors=[u'a-x', u'a-y', u'a-z', u'a-mag',u'g-x',u'g-y',u'g-z',u'g-mag']
# col_sensors=[ u'a-mag',u'g-mag']
X_train_test=df_data.loc[:,col_sensors][:rounded_data_idx].to_numpy().T.reshape(N_samples,8,200,1)
y_train_test_vec=df_data.loc[:,'id'][:rounded_data_idx].values[0::Ns]

from random import shuffle as shuffle_array
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit

# For an 80-20 split
# skf =StratifiedShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
skf=StratifiedKFold(n_splits=5,shuffle=True)
ind_split=skf.split(np.zeros(N_samples), y_train_test_vec)
train_indices=ind_split.__next__()[0]
test_indices=ind_split.__next__()[1]
shuffle_array(train_indices)
shuffle_array(test_indices)

y_train=y_train_test_vec[train_indices]
y_test=y_train_test_vec[test_indices]

X_train=X_train_test[train_indices,:,:,:]
X_test=X_train_test[test_indices,:,:,:]

In [ ]:
nb_classes=4
# convert class vectors to binary class matrices
#Note that we need to subtract 1 from the user_label to get class label
# that begin at 0
Y_train = np_utils.to_categorical(y_train-1, nb_classes)
Y_test = np_utils.to_categorical(y_test-1, nb_classes)
print(X_train.shape)
print(X_test.shape)

(248, 8, 200, 1)
(62, 8, 200, 1)


In [ ]:
# cnn lstm model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import to_categorical
from matplotlib import pyplot


In [ ]:
def IDNet_cnn_keras(weights_path=None, img_rows=8, img_cols=200):
    nb_classes = 4
    # size of pooling area for max pooling
    nb_pool = 3
    ###############################################################
    ###############################################################
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'), input_shape=(img_rows,img_cols,1)))
    print(model.output_shape)
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
    print(model.output_shape)
    model.add(TimeDistributed(Dropout(0.5)))
    print(model.output_shape)
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    print(model.output_shape)
    model.add(TimeDistributed(Flatten()))
    print(model.output_shape)
    model.add(LSTM(100))
    print(model.output_shape)
    model.add(Dropout(0.5))
    print(model.output_shape)
    model.add(Dense(100, activation='relu'))
    print(model.output_shape)
    model.add(Dense(nb_classes, activation='softmax'))
    print(model.output_shape)
    return model
model = IDNet_cnn_keras()
model.compile(loss='categorical_crossentropy',optimizer='Adadelta',metrics=['accuracy'])

History=model.fit(X_train, Y_train, batch_size=128,validation_split=0.125, epochs=100,verbose=1)
score = model.evaluate(X_test, Y_test, verbose=1)
model.save("/content/drive/My Drive/file h5/cnnLSTM100-4 ngươingươiclass.h5")
print('Test score:', score[0])
print('Test accuracy:', score[1])

(None, 8, 198, 64)
(None, 8, 196, 64)
(None, 8, 196, 64)
(None, 8, 98, 64)
(None, 8, 6272)
(None, 100)
(None, 100)
(None, 100)
(None, 4)
Train on 217 samples, validate on 31 samples
Epoch 1/100
217/217 [==============================] - 3s 15ms/step - loss: 1.4266 - accuracy: 0.2442 - val_loss: 1.4352 - val_accuracy: 0.2903
Epoch 2/100
217/217 [==============================] - 2s 10ms/step - loss: 1.3182 - accuracy: 0.4009 - val_loss: 1.4034 - val_accuracy: 0.3871
Epoch 3/100
217/217 [==============================] - 2s 10ms/step - loss: 1.2378 - accuracy: 0.4286 - val_loss: 1.2879 - val_accuracy: 0.3871
Epoch 4/100
217/217 [==============================] - 2s 10ms/step - loss: 1.1378 - accuracy: 0.5069 - val_loss: 1.1051 - val_accuracy: 0.5484
Epoch 5/100
217/217 [==============================] - 2s 11ms/step - loss: 1.0569 - accuracy: 0.5668 - val_loss: 1.0222 - val_accuracy: 0.5806
Epoch 6/100
217/217 [==============================] - 2s 10ms/step - loss: 1.0468 - accuracy: 0.5

In [ ]:

df_data=pd.read_csv('/content/drive/My Drive/bạn phương.csv')
Ns = 200
N_samples=df_data.shape[0]//Ns
rounded_data_idx = N_samples*Ns
# col_sensors=[ u'a-mag', u'g-mag']
col_sensors=[u'a-x', u'a-y', u'a-z', u'a-mag',u'g-x', u'g-y', u'g-z', u'g-mag']
X_new=df_data.loc[:,col_sensors][:rounded_data_idx].to_numpy().T.reshape(N_samples,8,Ns,1)
ynew = model.predict_classes(X_new)
import numpy as np
unique_elements, counts_elements = np.unique(ynew, return_counts=True)
print(np.asarray(np.argmax(counts_elements)))



0


In [ ]:
ynew

array([0, 0, 0, 3, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
from sklearn.preprocessing import MinMaxScaler
df_data=pd.read_csv('/content/drive/My Drive/truong.csv')
Ns = 200
N_samples=df_data.shape[0]//Ns
rounded_data_idx = N_samples*Ns
# col_sensors=[ u'a-mag', u'g-mag']
col_sensors=[u'a-x', u'a-y', u'a-z', u'a-mag',u'g-x', u'g-y', u'g-z', u'g-mag']
X_new=df_data.loc[:,col_sensors][:rounded_data_idx].to_numpy().T.reshape(N_samples,8,Ns,1)
ynew = model.predict_classes(X_new)
import numpy as np
unique_elements, counts_elements = np.unique(ynew, return_counts=True)
print(np.asarray(np.argmax(counts_elements)))

1


In [ ]:
from sklearn.preprocessing import MinMaxScaler
df_data=pd.read_csv('/content/drive/My Drive/phuong.csv')
df_data.shape
Ns = 200
N_samples=df_data.shape[0]//Ns
rounded_data_idx = N_samples*Ns
col_sensors=[u'a-x', u'a-y', u'a-z', u'a-mag',u'g-x', u'g-y', u'g-z', u'g-mag']
X_new=df_data.loc[:,col_sensors][:rounded_data_idx].to_numpy().T.reshape(N_samples,8,Ns,1)
ynew = model.predict_classes(X_new)
# print("X=%s, Predicted=%s" % (X_new[0], ynew[0]))
import numpy as np
unique_elements, counts_elements = np.unique(ynew, return_counts=True)
print(np.asarray(np.argmax(counts_elements)))


2
